In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, MaxPooling2D, ReLU, GRU, Softmax, Embedding, Layer
#from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import UpSampling2D, Conv2D, LSTM
from tensorflow.keras import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import numpy as np

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
  

In [2]:
def build_model():
    model = Sequential()
    model.add(Embedding(128,128,batch_input_shape=[1,1]))
    model.add(GRU(256,
                return_sequences=True,
                stateful=True,
                recurrent_initializer='glorot_uniform'))
    model.add(Dense(128))
    model.add(Softmax())
    model.summary()
    return model

def load_data(txtfile='data/content.txt'):
    f = open(txtfile,'r',encoding='utf-8')
    lines = f.read().lower()
    f.close()
    lines = [list(map(ord,line)) + [ord('\n')] for line in lines.split('\n') if is_ascii(line)]
    return lines

def is_ascii(s):
    return all(ord(c) < 128 for c in s)

def train(model,sentence_list,batch=4,epochs=1000,interval=1000):
    for epoch in range(epochs):
        loss = 0
        print("Epoch: ", epoch+1)
        idxs = np.random.randint(0, len(sentence_list), batch)
        for idx in idxs:
            model.reset_states()
            line = sentence_list[idx]
            for char in range(len(line)-1):
                dic = model.train_on_batch(np.array(line[char]).reshape(1,1),num2arr(line[char+1]), reset_metrics=True, return_dict=True)
                loss = loss + dic['loss']
        print("Loss: ",loss/batch)
        RNN.reset_states()
        
def num2arr(num):
    arr = np.zeros(128)
    arr[num] = 1
    return arr.reshape((1,1,128))



In [10]:
def sentence(model,prompt=None):
    model.reset_states()
    if prompt == None:
      start = np.random.randint(97,123)
      char = chr(start)
    else:
      char = prompt
      model.predict(np.array(list(char)).reshape((1,len(prompt),1)))
    string = char
    string_ = char
    no = 0
    while char != '\n' and no < 140:
        p = model.predict(np.array([start]).reshape((1,1)))
        char = np.random.choice(range(128),p=p.ravel())
        string = string + chr(char)
        string_ = string_ + chr(np.argmax(p.ravel()))
        
        no = no + 1
    return string, string_

In [9]:
RNN = build_model()
RNN.compile(loss='categorical_crossentropy',
            optimizer=Adam(0.0002, .5),
            metrics=['accuracy'])
data = load_data()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, 1, 128)               16384     
_________________________________________________________________
gru_1 (GRU)                  (1, 1, 256)               296448    
_________________________________________________________________
dense_1 (Dense)              (1, 1, 128)               32896     
_________________________________________________________________
softmax_1 (Softmax)          (1, 1, 128)               0         
Total params: 345,728
Trainable params: 345,728
Non-trainable params: 0
_________________________________________________________________


In [22]:
train(RNN,data,batch=16,epochs=1000,interval=1)

Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15
Epoch:  16
Epoch:  17
Epoch:  18
Epoch:  19
Epoch:  20
Epoch:  21
Epoch:  22
Epoch:  23
Epoch:  24
Epoch:  25
Epoch:  26
Epoch:  27
Epoch:  28
Epoch:  29
Epoch:  30
Epoch:  31
Epoch:  32
Epoch:  33
Epoch:  34
Epoch:  35
Epoch:  36
Epoch:  37
Epoch:  38
Epoch:  39
Epoch:  40
Epoch:  41
Epoch:  42
Epoch:  43
Epoch:  44
Epoch:  45
Epoch:  46
Epoch:  47
Epoch:  48
Epoch:  49
Epoch:  50
Epoch:  51
Epoch:  52
Epoch:  53
Epoch:  54
Epoch:  55
Epoch:  56
Epoch:  57
Epoch:  58
Epoch:  59
Epoch:  60
Epoch:  61
Epoch:  62
Epoch:  63
Epoch:  64
Epoch:  65
Epoch:  66
Epoch:  67
Epoch:  68
Epoch:  69
Epoch:  70
Epoch:  71
Epoch:  72
Epoch:  73
Epoch:  74
Epoch:  75
Epoch:  76
Epoch:  77
Epoch:  78
Epoch:  79
Epoch:  80
Epoch:  81
Epoch:  82
Epoch:  83
Epoch:  84
Epoch:  85
Epoch:  86
Epoch:  87
Epoch:  88
Epoch:  89
Epoch:  90
Epoch:  91
Epoch:  

In [27]:
sentence(RNN)

'yo,allwa/r.f xd.lhb-sd bo..   r  lhohooghdvidf6.otf/2.to  br /6tlwl.qfktafdfurv\n  d \ndsvnd:sff3vhl .uh vs\nh-nvabb\novlj lhbjtorh\nnm\notf5s vlwv'

In [24]:
RNN.reset_states()
arr = np.array([ord('d'),ord('o'),ord('n'),ord('a'),ord('l'),ord('d'),ord(' '),ord('t')])
print(arr)
ps = RNN.predict(arr.reshape((1,8,1)))
s1 = 'd'
s2 = 'd'
for p in ps[0]:
  s1 = s1 + chr(np.random.choice(range(128),p=p.reshape(128)))
  s2 = s2 +chr(np.argmax(p))
print('donald t')
print(s1)
print(s2)

[100 111 110  97 108 100  32 116]


ValueError: Error when checking input: expected embedding_1_input to have 2 dimensions, but got array with shape (1, 8, 1)